In [57]:
import numpy as np
import torch
from torch import nn
import pandas as pd
from sklearn.model_selection import train_test_split

In [58]:
data = pd.read_csv("train.csv")
y = data['SalePrice']
X = data.drop('SalePrice', axis=1)
X.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [59]:
X.drop('Id', axis=1)
numeric_cols = list(X.select_dtypes(include=np.number).columns)
X_num = X[numeric_cols]
X_num.columns = X[numeric_cols].columns
X_num.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,548,0,61,0,0,0,0,0,2,2008
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,460,298,0,0,0,0,0,0,5,2007
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,608,0,42,0,0,0,0,0,9,2008
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,642,0,35,272,0,0,0,0,2,2006
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,836,192,84,0,0,0,0,0,12,2008


## First model will use only numeric values and missing values will be imputed with SimpleImputer

In [60]:
from sklearn.impute import SimpleImputer

my_imputer = SimpleImputer()

def get_train_test_data(X, y, test_size):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    
    imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
    imputed_X_test = pd.DataFrame(my_imputer.transform(X_test))
    
    imputed_X_train.columns = X_train.columns
    imputed_X_test.columns = X_test.columns
    
    X_train, X_test = imputed_X_train, imputed_X_test
    
    X_train = torch.tensor(X_train.values, dtype=torch.float32)
    X_test = torch.tensor(X_test.values, dtype=torch.float32)
    y_train = torch.tensor(y_train.values, dtype=torch.float32)
    y_test = torch.tensor(y_test.values, dtype=torch.float32)
    return X_train, X_test, y_train, y_test

In [93]:
X_train, X_test, y_train, y_test = get_train_test_data(X_num, y, test_size=0.2)
X_num = pd.DataFrame(my_imputer.transform(X_num))

In [99]:
class ModelV1(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(in_features=in_features, out_features=1),
            # nn.Linear(in_features=10, out_features=10),
            # nn.Linear(in_features=10, out_features=1)
        )
        
    def forward(self, x):
        return self.layers(x)

In [108]:
model1 = ModelV1(in_features=37)

loss_fn = nn.L1Loss()
optimizer = torch.optim.SGD(params=model1.parameters(), lr=0.0001)

In [109]:
model1.eval()
y_pred = model1(X_train).squeeze()
y_pred

tensor([ 843.7307,  915.1597, 1170.9192,  ..., 1338.3646, 1107.5521,
        1050.4900], grad_fn=<SqueezeBackward0>)

In [110]:
def train_test(model:torch.nn.Module, loss_fn:torch.nn.Module, optimizer:torch.optim.Optimizer, epochs:int, X_train, X_test, y_train, y_test):
    
    for epoch in range(epochs):
        model.train()
        y_preds = model(X_train).squeeze()
        loss = loss_fn(y_preds, y_train)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        model.eval()
        with (torch.inference_mode()):
            test_pred = model(X_test).squeeze()
            test_loss = loss_fn(test_pred, y_test)
        if epoch % 50 == 0:
            print("----------------------------")
            print(f"Epoch {epoch}:")
            print(f"Train loss: {loss} | Test loss: {test_loss}")

In [111]:
train_test(model=model1, loss_fn=loss_fn, optimizer=optimizer, epochs=2000, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)

----------------------------
Epoch 0:
Train loss: 178740.296875 | Test loss: 171537.421875
----------------------------
Epoch 50:
Train loss: 59481.9921875 | Test loss: 57938.1640625
----------------------------
Epoch 100:
Train loss: 54991.64453125 | Test loss: 54821.1171875
----------------------------
Epoch 150:
Train loss: 51404.84765625 | Test loss: 52224.13671875
----------------------------
Epoch 200:
Train loss: 48755.9296875 | Test loss: 50324.21484375
----------------------------
Epoch 250:
Train loss: 46787.62890625 | Test loss: 48741.765625
----------------------------
Epoch 300:
Train loss: 45368.17578125 | Test loss: 47503.1328125
----------------------------
Epoch 350:
Train loss: 44312.11328125 | Test loss: 46636.7578125
----------------------------
Epoch 400:
Train loss: 43373.04296875 | Test loss: 45797.54296875
----------------------------
Epoch 450:
Train loss: 42592.984375 | Test loss: 45001.95703125
----------------------------
Epoch 500:
Train loss: 41882.8828125

## This results are not great, let's figure out which 5 data columns have the most corelation with SalePrice and use only them

In [112]:
from sklearn.feature_selection import mutual_info_regression

def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

In [122]:
discrete_features = X_num.dtypes == int

mi_scores = make_mi_scores(X_num, y, discrete_features=discrete_features)
mi_scores[:7]

4     0.559843
16    0.482892
12    0.366936
26    0.366470
27    0.364180
6     0.363803
13    0.310731
Name: MI Scores, dtype: float64

## Let's now use only this 7 features in a Model and see if there is any difference in performance

In [123]:
model2 = ModelV1(in_features=7)
X_num_2 = X_num[list(mi_scores[:7].index)]
X_train, X_test, y_train, y_test = get_train_test_data(X_num_2, y, test_size=0.2)
optimizer = torch.optim.SGD(params=model2.parameters(), lr=0.001)

In [124]:
train_test(model=model2, loss_fn=loss_fn, optimizer=optimizer, epochs=2000, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)

----------------------------
Epoch 0:
Train loss: 180676.6875 | Test loss: 175690.125
----------------------------
Epoch 50:
Train loss: 37957.44140625 | Test loss: 40463.37109375
----------------------------
Epoch 100:
Train loss: 36233.81640625 | Test loss: 38626.58203125
----------------------------
Epoch 150:
Train loss: 34877.3515625 | Test loss: 37109.23828125
----------------------------
Epoch 200:
Train loss: 33828.75390625 | Test loss: 35856.296875
----------------------------
Epoch 250:
Train loss: 33010.765625 | Test loss: 34834.5625
----------------------------
Epoch 300:
Train loss: 32319.75 | Test loss: 33934.6328125
----------------------------
Epoch 350:
Train loss: 31775.119140625 | Test loss: 33301.6875
----------------------------
Epoch 400:
Train loss: 31368.0234375 | Test loss: 32761.423828125
----------------------------
Epoch 450:
Train loss: 31063.201171875 | Test loss: 32358.5078125
----------------------------
Epoch 500:
Train loss: 30808.787109375 | Test loss